# Workflow for establishing consolidated labelling

This workflow is for working with files that are imported from Labelstudio with annotations from multiple annotators.

## I. Consistency with the benchmark setup

* Check that labellings are consitent with the benchmark setup
* Check that labellings of different persons overlap in predicted way

In [5]:
import json

filename = 'labelled_data/multiple_annotators.json'

#check it can be opened and json loaded
with open(filename,'r',encoding='UTF-8') as f:
    data = json.load(f)

In [7]:
for task in data:
    #we have multiple annotations per text in this setup
    assert len(task['annotations']) > 1
    #check that the task has been annotated by someone
    assert task['annotations'][0]['completed_by'] > 0

## II. Create a labelling task for consolidating conflicting labellings

* Create a labelling task where the first instances are inconsitent followed by consistent entries
* So that a labeller can go from top to last inconsitency and be done with the task

In [36]:
conflicting_tasks = []
non_conflicting_tasks = []

for task in data:
    conflicting = False
    choice = None
    #todo check that this loop works correctly
    for annotation in task['annotations']:
        if 'choices' in annotation['result'][-1]['value']:
            if choice is None:
                choice = annotation['result'][-1]['value']['choices'][0]
            elif annotation['result'][-1]['value']['choices'][0] != choice:
                conflicting = True
    if conflicting:
        conflicting_tasks.append(task)
    else:
        non_conflicting_tasks.append(task)

In [24]:
new_tasks = []
new_tasks.extend(conflicting_tasks)
new_tasks.extend(non_conflicting_tasks)

In [28]:
with open('ls_task.json','w',encoding='UTF-8') as f:
    f.write(json.dumps(new_tasks))